This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the pseudo-bulk analysis for the complex model with interaction terms that was run on the patient-level data. This script contains the code that was run on the full data and does not test for smoking associations.

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_pseudo_nUMIoffset_testInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [10]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [11]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [12]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Misharin', 'Spence', 'Rawlins', 'Schultze/Falk', 'Linnarsson']

In [13]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [14]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [15]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [16]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [17]:
# Rename smoking status covariate
adata.obs['smoking_status'] = adata.obs.smoked_boolean

In [18]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Xavier/Regev                25552
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

309

185

# Check the data

In [19]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [20]:
# Check if any non-integer data in a particular dataset
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are (or nearly are) integers

In [21]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
55.50    27176
20.00    25362
51.00    24766
72.50    24366
30.00    24095
58.00    23723
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
45.50    16351
56.00    16011
23.00    15181
41.00    14943
49.00    14225
57.50    13840
55.00    13733
68.00    11852
65.00    11404
75.00    11243
45.00    10939
47.00    10278
52.50    10133
21.00     9430
67.00     9278
27.00     8693
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
62.00     6228
38.00     5629
73.00     5251
24.00     4433
60.00     4231
62.50     4073
26.00     4066
61.00     3922
50.00     3831
32.50     3772
52.00     3765
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
53.00     1592
37.00     1240
48.00     1153
81.00     1145
25.00      823
70.00     

female    583926
male      512678
Name: sex, dtype: int64

# Fit models and perform DE

In [22]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium',
 'Endothelial-like',
 'Granulocytes']

Calculate DE genes per cluster.

In [23]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

# Generate pseudobulks

In [24]:
for gene in adata.var_names:
    adata.obs[gene] = adata[:,gene].X.A

In [25]:
dat_pseudo = adata.obs.groupby(['donor', 'ann_level_2']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo = sc.AnnData(dat_pseudo[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [26]:
adata_pseudo.obs.head()

,donor,ann_level_2,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,1_Endothelial,6307.156863,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,Airway epithelium,4948.705263,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Alveolar epithelium,4268.209154,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Fibroblast lineage,5888.260870,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Lymphatics,5503.117647,47.0,True,female,Eils/Conrad/Kreuter


In [27]:
adata_pseudo.obs['total_counts_scaled'] = adata_pseudo.obs['total_counts']/adata_pseudo.obs['total_counts'].mean()

In [28]:
# Get interquartile range for ages to test
adata_pseudo.obs.groupby(['donor']).agg({'age':'first'}).age.quantile([0.25,0.5,0.75])

0.25    32.0
0.50    52.0
0.75    62.5
Name: age, dtype: float64

In [29]:
formula = "1 + sex + age + sex:age + dataset"
tested_coef = ["sex[T.male]", "age"]
dmat = de.utils.design_matrix(
    data=adata,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)

to_test = dict()
to_test['age'] = [32,62]
to_test['sex[T.male]'] = [0,1]

dmat[1]

['Intercept',
 'sex[T.male]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Schultze]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'dataset[T.Xavier/Regev]',
 'age',
 'sex[T.male]:age']

### Function definition to test effect sizes at particular covariate values

In [30]:
def calc_effects(dmat, cov_mat, params, effect, coefs):
    from patsy.design_info import DesignMatrix
    from diffxpy.api.stats import wald_test_chisq
    
    dmat_cond = isinstance(dmat, tuple) and isinstance(dmat[0], DesignMatrix)

    if not dmat_cond:
        raise ValueError("`dmat` should be a patsy output Design Matrix.")

    effect_list = ['sex[T.male]', 'age', 'smoking_status[T.True]']
    if not effect in effect_list:
        raise ValueError(f'{effect} is not one of: '
                         f'{effect_list}')

    if not isinstance(coefs, dict):
        raise TypeError('`coefs` should contain a dictionary of coefficients '
                        'where the effects should be evaluated.')
    
    ## Note: this is only correct when 3 covariates are tested in combinations
    #if np.sum([coef in coefs for coef in effect_list]) < 2:
    #    raise ValueError('The `coefs` dict must contain values for the two '
    #                     'coefficient not tested in:'
    #                     f'{effect_list}')
        
    if 'smoking_status[T.True]' in coefs and coefs['smoking_status[T.True]'] not in [0,1]:
        raise ValueError('Smoking status should be encoded as 0 or 1.')

    if 'sex[T.male]' in coefs and coefs['sex[T.male]'] not in [0,1]:
        raise ValueError('Sex should be encoded as 0 or 1.')

    if 'age' in coefs and not (isinstance(coefs['age'], float) or isinstance(coefs['age'], int)):
        raise ValueError('Age should be a numerical value.')

    coef_list = []

    for term in dmat[1]:
        if effect not in term:
            coef_list.append(0)
        elif term == effect:
            coef_list.append(1)
        else:
            t_list = term.split(':')
            t_list.remove(effect)
            coef_list.append(coefs[t_list[0]])

    C = np.array(coef_list)
    val = np.matmul(C,np.array(params))
    stderr = np.sqrt(np.matmul(np.matmul(C.T,cov_mat),C))
    pval = wald_test_chisq(np.array([val]).reshape(1,1), np.array([stderr**2]).reshape(1,1,1))[0]
    return (val, stderr, pval)

## Poisson GLM

In [31]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo[adata_pseudo.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['smoking_status'].value_counts())
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_pseudo_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i], #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                coef_vals = {iter_coefs[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                case = iter_coefs[0]+':'+str(c1)
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:32','32yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
False    66
True     55
nan      21
Name: smoking_status, dtype: int64
male      76
female    66
Name: sex, dtype: int64
Testing 3 genes...

In cluster Airway epithelium:
False    90
True     71
nan      20
Name: smoking_status, dtype: int64
male      97
female    84
Name: sex, dtype: int64
Testing 3 genes...

In cluster Alveolar epithelium:
False    67
True     39
nan      14
Name: smoking_status, dtype: int64
male      64
female    56
Name: sex, dtype: int64
Testing 3 genes...

In cluster Lymphoid:
False    72
True     62
nan      21
Name: smoking_status, dtype: int64
male      82
female    73
Name: sex, dtype: int64
Testing 3 genes...

In cluster Fibroblast lineage:
False    54
True     40
nan      14
Name: smoking_status, dtype: int64
male      59
female    49
Name: sex, dtype: int64
Testing 3 genes...

In cluster Blood vessels:
False    48
True     31
nan      13
Name: smoking_status, dtype: int64
male      54
female    38
Name: sex, dtype: int64
Testing 3 gene

## Inspect some results

In [32]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium', 'Granulocytes'])

In [33]:
full_res_lvl2_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],32yr,0.298059,9.510353,0.974998,0.975935
1,ACE2,Myeloid,sex[T.male],62yr,0.288443,9.562100,0.975935,0.975935
2,ACE2,Myeloid,age,F,-0.018231,0.277863,0.947689,0.975935
3,ACE2,Myeloid,age,M,-0.018551,0.389273,0.961991,0.975935
4,TMPRSS2,Myeloid,sex[T.male],32yr,0.995246,2.152080,0.643753,0.975935
5,TMPRSS2,Myeloid,sex[T.male],62yr,0.603703,1.824983,0.740796,0.975935
6,TMPRSS2,Myeloid,age,F,0.006562,0.048913,0.893286,0.975935
7,TMPRSS2,Myeloid,age,M,-0.006490,0.091134,0.943228,0.975935
8,CTSL,Myeloid,sex[T.male],32yr,-0.121609,0.142934,0.394877,0.975935
9,CTSL,Myeloid,sex[T.male],62yr,0.056677,0.143601,0.693077,0.975935


In [34]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],32yr,0.298059,9.510353,0.974998,0.975935
1,ACE2,Myeloid,sex[T.male],62yr,0.288443,9.562100,0.975935,0.975935
2,ACE2,Myeloid,age,F,-0.018231,0.277863,0.947689,0.975935
3,ACE2,Myeloid,age,M,-0.018551,0.389273,0.961991,0.975935
12,ACE2,Airway epithelium,sex[T.male],32yr,0.050874,1.500049,0.972945,0.992206
13,ACE2,Airway epithelium,sex[T.male],62yr,0.013722,1.404795,0.992206,0.992206
14,ACE2,Airway epithelium,age,F,-0.004533,0.055162,0.934507,0.992206
15,ACE2,Airway epithelium,age,M,-0.005771,0.050994,0.909889,0.992206
24,ACE2,Alveolar epithelium,sex[T.male],32yr,0.555897,2.934558,0.849755,0.992571
25,ACE2,Alveolar epithelium,sex[T.male],62yr,-0.083306,2.588266,0.974324,0.992571


In [35]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
4,TMPRSS2,Myeloid,sex[T.male],32yr,0.995246,2.152080,0.643753,0.975935
5,TMPRSS2,Myeloid,sex[T.male],62yr,0.603703,1.824983,0.740796,0.975935
6,TMPRSS2,Myeloid,age,F,0.006562,0.048913,0.893286,0.975935
7,TMPRSS2,Myeloid,age,M,-0.006490,0.091134,0.943228,0.975935
16,TMPRSS2,Airway epithelium,sex[T.male],32yr,0.256681,0.359212,0.474876,0.992206
17,TMPRSS2,Airway epithelium,sex[T.male],62yr,0.195109,0.363213,0.591147,0.992206
18,TMPRSS2,Airway epithelium,age,F,-0.003329,0.012262,0.786026,0.992206
19,TMPRSS2,Airway epithelium,age,M,-0.005381,0.011938,0.652148,0.992206
28,TMPRSS2,Alveolar epithelium,sex[T.male],32yr,0.236332,0.290944,0.416622,0.992571
29,TMPRSS2,Alveolar epithelium,sex[T.male],62yr,0.161888,0.293829,0.581661,0.992571


# Level 3 annotation

In [36]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [37]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 895743 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'ACE2', 'TMPRSS2', 'CTSL'

185

309

## Generate pseudobulk

In [38]:
for gene in adata_sub.var_names:
    adata_sub.obs[gene] = adata_sub[:,gene].X.A

Trying to set attribute `.obs` of view, copying.


In [39]:
dat_pseudo_sub = adata_sub.obs.groupby(['donor', 'ann_level_3']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo_sub = sc.AnnData(dat_pseudo_sub[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo_sub.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [40]:
adata_pseudo_sub.obs.head()

,donor,ann_level_3,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,AT1,3709.913265,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,AT2,4453.519898,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Lymphatic EC,5503.117647,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Monocytes,3339.083948,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Multiciliated lineage,6000.121212,47.0,True,female,Eils/Conrad/Kreuter


In [41]:
adata_pseudo_sub.obs['total_counts_scaled'] = adata_pseudo_sub.obs['total_counts']/adata_pseudo_sub.obs['total_counts'].mean()

## Poisson GLM

First check if there are any datasets with only 1 sex or 1 smoking status, which would make the model overparameterized (not full rank).

In [42]:
np.any(adata_pseudo_tmp.obs.smoking_status.value_counts() == 1)

True

In [43]:
np.any(pd.crosstab(adata_pseudo_tmp.obs.smoking_status, adata_pseudo_tmp.obs.sex) == 1)

True

In [44]:
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [45]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo_sub[adata_pseudo_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['smoking_status'].value_counts())
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_pseudo_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
    if np.any(adata_pseudo_tmp.obs.sex.value_counts()==1):
        print(f'{clust} only has 1 male or 1 female sample.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp,
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i],
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                coef_vals = {iter_coefs[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                case = iter_coefs[0]+':'+str(c1)
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:32','32yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)
                
    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
False    49
True     40
nan      21
Name: smoking_status, dtype: int64
male      64
female    46
Name: sex, dtype: int64
Testing 3 genes...

In cluster AT2:
False    66
True     38
nan      14
Name: smoking_status, dtype: int64
male      62
female    56
Name: sex, dtype: int64
Testing 3 genes...

In cluster Basal:
False    53
True     52
nan       5
Name: smoking_status, dtype: int64
male      62
female    48
Name: sex, dtype: int64
Testing 3 genes...

In cluster T cell lineage:
False    51
True     46
nan      21
Name: smoking_status, dtype: int64
male      65
female    53
Name: sex, dtype: int64
Testing 3 genes...

In cluster Monocytes:
False    57
True     39
nan      11
Name: smoking_status, dtype: int64
male      58
female    49
Name: sex, dtype: int64
Testing 3 genes...

In cluster Multiciliated lineage:
False    88
True     69
nan      13
Name: smoking_status, dtype: int64
male      91
female    79
Name: sex, dtype: int64
Testing 3 genes...

In cluster AT

## Inspect some results

In [46]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'AT2', 'Basal', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'AT1', 'Submucosal Secretory', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Myofibroblasts', 'MDC', 'Airway smooth muscle'])

In [47]:
full_res_lvl3_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],32yr,-0.218032,12.889231,0.986504,0.994505
1,ACE2,Macrophages,sex[T.male],62yr,-1.553912,12.425535,0.900478,0.994505
2,ACE2,Macrophages,age,F,0.016025,0.348604,0.963335,0.994505
3,ACE2,Macrophages,age,M,-0.028505,0.477177,0.952366,0.994505
4,TMPRSS2,Macrophages,sex[T.male],32yr,-0.091657,3.270424,0.977641,0.994505
5,TMPRSS2,Macrophages,sex[T.male],62yr,-0.234202,2.856403,0.934653,0.994505
6,TMPRSS2,Macrophages,age,F,0.000637,0.092524,0.994505,0.994505
7,TMPRSS2,Macrophages,age,M,-0.004114,0.113119,0.970987,0.994505
8,CTSL,Macrophages,sex[T.male],32yr,-0.060452,0.136767,0.658481,0.994505
9,CTSL,Macrophages,sex[T.male],62yr,-0.173228,0.124193,0.163066,0.839500


In [48]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],32yr,-0.218032,12.889231,0.986504,0.994505
1,ACE2,Macrophages,sex[T.male],62yr,-1.553912,12.425535,0.900478,0.994505
2,ACE2,Macrophages,age,F,0.016025,0.348604,0.963335,0.994505
3,ACE2,Macrophages,age,M,-0.028505,0.477177,0.952366,0.994505
12,ACE2,AT2,sex[T.male],32yr,0.687516,2.639726,0.794517,0.972298
13,ACE2,AT2,sex[T.male],62yr,-0.150136,2.454907,0.951234,0.972298
14,ACE2,AT2,age,F,0.019501,0.086401,0.821430,0.972298
15,ACE2,AT2,age,M,-0.008421,0.076749,0.912635,0.972298
24,ACE2,Basal,sex[T.male],32yr,-0.387014,2.282738,0.865373,0.959020
25,ACE2,Basal,sex[T.male],62yr,0.544868,1.984062,0.783606,0.959020


In [49]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
4,TMPRSS2,Macrophages,sex[T.male],32yr,-0.091657,3.270424,0.977641,0.994505
5,TMPRSS2,Macrophages,sex[T.male],62yr,-0.234202,2.856403,0.934653,0.994505
6,TMPRSS2,Macrophages,age,F,0.000637,0.092524,0.994505,0.994505
7,TMPRSS2,Macrophages,age,M,-0.004114,0.113119,0.970987,0.994505
16,TMPRSS2,AT2,sex[T.male],32yr,0.221455,0.276070,0.422456,0.972298
17,TMPRSS2,AT2,sex[T.male],62yr,0.156586,0.281139,0.577549,0.972298
18,TMPRSS2,AT2,age,F,0.009491,0.008997,0.291503,0.972298
19,TMPRSS2,AT2,age,M,0.007328,0.008778,0.403785,0.972298
28,TMPRSS2,Basal,sex[T.male],32yr,0.454407,0.877209,0.604448,0.959020
29,TMPRSS2,Basal,sex[T.male],62yr,0.336340,0.678193,0.619940,0.959020


# Store results

In [55]:
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [56]:
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')